**Para importar y utilizar las funciones del módulo del proyecto, es necesario instalarlo previamente. Sigue las instrucciones detalladas en el archivo install.md, donde se describen los pasos necesarios para completar la instalación correctamente.**

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import os
import re
# Importaciones específicas del proyecto
from energy_consumption_architecture.utils.paths import data_dir, data_raw_dir
from energy_consumption_architecture.clustering_utils import *
from energy_consumption_architecture.dataset import load_all_series
from energy_consumption_architecture.regresion_utils import regression_pipeline_for_clusters,evaluate_all_clusters
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# Procesamiento de  datos 

**Antes de proceder a los pasos siguientes, el usuario debe realizar un preprocesamiento adecuado de las series de tiempo. Esto incluye tareas como la limpieza de datos, el tratamiento de valores nulos y el manejo de cualquier peculiaridad específica de las series de tiempo. Estas acciones son esenciales, ya que el análisis de series de tiempo puede variar significativamente según la naturaleza de los datos y el contexto en el que se utilicen.**

**Dado que las necesidades de preprocesamiento dependen del caso de uso, los objetivos y las características intrínsecas de las series de tiempo, este proceso no se estandarizará dentro de esta arquitectura. En su lugar, se deja al criterio del usuario la implementación de estas tareas, adaptándolas a las particularidades de cada conjunto de datos.** 

## Cargar datos 


Al clonar el proyecto, debe estar presente la carpeta `data`. Si esta carpeta no existe, será necesario crearla manualmente, ya que es fundamental para la organización de los datos del proyecto.

La función `data_dir` se utiliza para gestionar las rutas hacia la carpeta `data` y sus subdirectorios. 

- Si se llama sin argumentos (`data_dir()`), la función devuelve la ruta principal de la carpeta `data`.  
- Si se pasa un nombre como argumento, por ejemplo, `data_dir("raw")`, la función construye y devuelve la ruta al subdirectorio especificado dentro de `data`.

In [6]:
# Especifica el directorio donde están los archivos CSV
carpeta=data_dir("raw")
# Obtén la lista de todos los archivos en la carpeta
archivos = os.listdir(carpeta)
archivos[:5]

['RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4A_USA_MD_BALTIMORE_Belleville-Scott.csv',
 'RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4A_USA_MD_BALTIMORE_Cahokia.csv',
 'RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4A_USA_MD_BALTIMORE_Carbondale-Southern.csv',
 'RefBldgFullServiceRestaurantNew2004_v1.3_7.1_5A_USA_IL_CHICAGO-OHARE_Aurora.Muni.csv',
 'RefBldgFullServiceRestaurantNew2004_v1.3_7.1_5A_USA_IL_CHICAGO-OHARE_Bloomington.csv']

La función `load_all_series` se utiliza para cargar múltiples series de tiempo desde archivos CSV y combinarlas en un único `DataFrame` para su análisis. 

- **Parámetros**: Recibe una lista con las rutas de los archivos y, opcionalmente, una lista de las columnas que se desean extraer. Si no se especifican las columnas, la función cargará todas las disponibles en cada archivo.
- **Validaciones**: Es importante asegurarse de que los archivos contengan las columnas especificadas, ya que de lo contrario se generará un error durante la carga.
- **Adiciones Automáticas**:
  - Asigna un identificador único (`ID`) a cada serie al momento de cargarla.
  - Extrae y almacena el nombre del archivo, útil si este describe información relevante sobre los datos.

In [41]:
# Define the columns to keep
columns_to_keep = [
    'Date/Time',
    'Cooling:Electricity [kW](Hourly)',
    'InteriorEquipment:Electricity [kW](Hourly)'
]

In [42]:
combined_df_filtered = load_all_series(archivos, columns_to_keep)

In [43]:
combined_df_filtered.head()

,Date/Time,Cooling:Electricity [kW](Hourly),InteriorEquipment:Electricity [kW](Hourly),file_name,series_id
0,01/01 01:00:00,0.000733,8.1892,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
1,01/01 02:00:00,0.000000,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
2,01/01 03:00:00,0.000000,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
3,01/01 04:00:00,0.000000,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
4,01/01 05:00:00,0.000000,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1


En este paso, es necesario asignar un nombre específico a la columna que contiene los *timestamps* de las series de tiempo. Se debe utilizar el nombre `"Date/Time"` para estandarizar y facilitar los pasos posteriores en el análisis.

Este proceso no se ha estandarizado completamente dentro de la arquitectura, ya que el tratamiento de las series de tiempo puede variar según las características del dataset. Por lo tanto, el usuario debe ajustar este paso para adaptarlo a su conjunto de datos, asegurándose de que la columna de *timestamps* sea convertida al tipo de dato `datetime`. Esto es fundamental para que las funciones posteriores trabajen correctamente con la serie temporal.

In [44]:
# Procesar la columna de fecha y tiempo
combined_df_filtered.rename(columns={"Date/Time": "Date/Time"}, inplace=True)
combined_df_filtered["Date/Time"] = '2004 ' + combined_df_filtered["Date/Time"]
date_format = '%Y %m/%d %H:%M:%S'
combined_df_filtered["Date/Time"] = pd.to_datetime(combined_df_filtered["Date/Time"], format=date_format, errors='coerce')

In [45]:
combined_df_filtered.head()

,Date/Time,Cooling:Electricity [kW](Hourly),InteriorEquipment:Electricity [kW](Hourly),file_name,series_id
0,2004-01-01 01:00:00,0.000733,8.1892,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
1,2004-01-01 02:00:00,0.000000,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
2,2004-01-01 03:00:00,0.000000,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
3,2004-01-01 04:00:00,0.000000,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
4,2004-01-01 05:00:00,0.000000,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1


## Obtener caracteristicas de las series de tiempo

Se extraen características representativas de las series temporales, específicamente la media y la desviación estándar de cada columna del DataFrame. Estas estadísticas resumen el comportamiento general de las series y proporcionan una base para el análisis y modelado posterior.

In [46]:
df_stats = calculate_statistics(combined_df_filtered)

In [47]:
df_stats.head()

,series_id,Cooling:Electricity [kW](Hourly)_mean,Cooling:Electricity [kW](Hourly)_std_dev,InteriorEquipment:Electricity [kW](Hourly)_mean,InteriorEquipment:Electricity [kW](Hourly)_std_dev
0,series_1,3.073887,6.226848,18.995908,7.265027
1,series_2,3.446951,6.596764,18.995908,7.265027
2,series_3,3.973240,7.146033,18.995908,7.265027
3,series_4,2.055950,4.688730,18.995908,7.265027
4,series_5,2.114192,4.797245,18.995908,7.265027


# Clustering

Esta función utiliza las características de las series de tiempo para evaluar cuál método de agrupación ofrece los mejores resultados. Se prueba la agrupación tanto con todas las características originales como con las reducidas mediante **PCA**. Se implementan tres algoritmos de clustering: **K-Means**, **DBSCAN**, y **Jerárquico**. Como salida, la función proporciona:

1. Una tabla con las métricas de evaluación para cada método.
2. El pipeline del proceso con el mejor rendimiento.
3. Los datos etiquetados con el grupo asignado a cada muestra.

In [48]:
# Ejecutar el pipeline automatizado
metrics, best_pipeline, clustered_data = automated_clustering_pipeline(df_stats)

In [49]:
# Resultados
print("Métricas de los métodos evaluados:")
metrics.sort_values(by="Combined Score",ascending=False)

Métricas de los métodos evaluados:


,Model,Silhouette Score,Davies-Bouldin Index,Num Clusters,PCA Applied,Silhouette Score Norm,Davies-Bouldin Index Norm,Combined Score
3,K-Means,0.838543,0.216405,4,True,1.000000,1.000000,1.000000
0,K-Means,0.835796,0.217897,4,False,0.987994,0.995679,0.991837
5,Hierarchical,0.791535,0.559966,2,True,0.794534,0.005481,0.400008
2,Hierarchical,0.789483,0.561860,2,False,0.785562,0.000000,0.392781
4,DBSCAN,0.612861,0.540670,5,True,0.013563,0.061338,0.037450
1,DBSCAN,0.609758,0.540998,5,False,0.000000,0.060390,0.030195


In [50]:
print("\nPipeline del mejor proceso:")
best_pipeline


Pipeline del mejor proceso:


Pipeline(steps=[('scaler', StandardScaler()),
                ('pca', PCA(n_components=np.int64(3))),
                ('clustering', KMeans(n_clusters=4, random_state=42))])

In [51]:
# Mostrar las primeras filas del DataFrame con clusters asignados
clustered_data.loc[:,["Cluster"]].value_counts()

Cluster
0          65
1           5
2           5
3           5
Name: count, dtype: int64

# Regresion

In [69]:
# Define the columns to keep
columns_to_keep = [
    'Date/Time',
    'Electricity:Facility [kW](Hourly)',
    'Fans:Electricity [kW](Hourly)',
    'Cooling:Electricity [kW](Hourly)',
    'Heating:Electricity [kW](Hourly)',
    'InteriorLights:Electricity [kW](Hourly)',
    'InteriorEquipment:Electricity [kW](Hourly)'
]
data_complete = load_all_series(archivos, columns_to_keep)
# Procesar la columna de fecha y tiempo
data_complete.rename(columns={"Date/Time": "Date/Time"}, inplace=True)
data_complete["Date/Time"] = '2004 ' + data_complete["Date/Time"]
date_format = '%Y %m/%d %H:%M:%S'
data_complete["Date/Time"] = pd.to_datetime(data_complete["Date/Time"], format=date_format, errors='coerce')

In [70]:
data_complete.head()

,Date/Time,Electricity:Facility [kW](Hourly),Fans:Electricity [kW](Hourly),Cooling:Electricity [kW](Hourly),Heating:Electricity [kW](Hourly),InteriorLights:Electricity [kW](Hourly),InteriorEquipment:Electricity [kW](Hourly),file_name,series_id
0,2004-01-01 01:00:00,22.453919,3.998243,0.000733,0.0,4.589925,8.1892,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
1,2004-01-01 02:00:00,14.637149,0.000000,0.000000,0.0,1.529975,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
2,2004-01-01 03:00:00,14.651183,0.000000,0.000000,0.0,1.529975,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
3,2004-01-01 04:00:00,14.657947,0.000000,0.000000,0.0,1.529975,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1
4,2004-01-01 05:00:00,14.806050,0.000000,0.000000,0.0,1.529975,7.4902,RefBldgFullServiceRestaurantNew2004_v1.3_7.1_4...,series_1


In [71]:
# Asegúrate de que el DataFrame `df_stats` contenga las etiquetas de cluster y el `series_id`
# Y que el DataFrame `data_complete` tenga el `series_id`

average_time_series_by_cluster = calculate_average_time_series_by_cluster(data_complete, clustered_data)

In [72]:
average_time_series_by_cluster.head()

,Cluster,Electricity:Facility [kW](Hourly),Fans:Electricity [kW](Hourly),Cooling:Electricity [kW](Hourly),Heating:Electricity [kW](Hourly),InteriorLights:Electricity [kW](Hourly),InteriorEquipment:Electricity [kW](Hourly)
Date/Time,,,,,,,
2004-01-01 01:00:00,0,47.210969,5.789775,3.010854,4.637630,6.918895,10.669373
2004-01-01 02:00:00,0,47.047291,5.946731,3.048947,4.450657,6.588343,10.525933
2004-01-01 03:00:00,0,45.619393,5.862989,2.883068,5.406697,5.084587,10.308935
2004-01-01 04:00:00,0,45.697195,6.186896,2.968095,4.801340,5.084587,10.297614
2004-01-01 05:00:00,0,47.030928,5.832209,2.942949,5.603789,5.182567,10.482863


In [73]:
target = 'Electricity:Facility [kW](Hourly)'

In [74]:
# Cargar modelos con configuraciones ajustadas
models = {
    "Linear Regression": LinearRegression(fit_intercept=True, n_jobs=-1),
    "Tree": DecisionTreeRegressor(max_depth=5, min_samples_split=5, random_state=42),
    "SVM": SVR(kernel='rbf', C=1.0, epsilon=0.1),
    "Random Forest": RandomForestRegressor(n_estimators=100, max_depth=5, min_samples_split=10, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, subsample=0.8, colsample_bytree=0.8, random_state=42)
}

In [75]:
metrics_df, best_pipelines = regression_pipeline_for_clusters(
    data=average_time_series_by_cluster,  # DataFrame con series de tiempo promedio
    target="Electricity:Facility [kW](Hourly)",  # Variable objetivo
    models=models,  # Diccionario de modelos de regresión
    test_size=0.2,  # Tamaño del conjunto de prueba
    threshold_ratio=1.5  # Relación máxima para detectar sobreentrenamiento
)


Processing Cluster 0





Processing Cluster 1





Processing Cluster 2





Processing Cluster 3







In [78]:
# Mostrar métricas de evaluación
print("Métricas de los modelos evaluados por cluster:")
metrics_df

Métricas de los modelos evaluados por cluster:


,Train RMSE,Train MAE,Train R2,Test RMSE,Test MAE,Test R2,Model Name,Cluster
0,3.763506,2.841202,0.983099,5.124020,3.953832,0.957409,Linear Regression,0
1,3.548263,2.687371,0.984977,4.054245,2.861092,0.973336,Tree,0
2,2.637984,1.756096,0.991696,3.556817,2.444444,0.979478,SVM,0
3,2.888434,2.209679,0.990045,3.334699,2.571090,0.981961,Random Forest,0
4,1.972782,1.398266,0.995356,2.637199,1.867744,0.988718,XGBoost,0
5,36.740954,31.211792,0.993652,42.556044,35.538846,0.986363,Linear Regression,1
6,63.605178,41.331628,0.980975,72.381552,50.592769,0.960551,Tree,1
7,53.211016,35.035966,0.986685,40.704668,31.691607,0.987524,SVM,1
8,51.737283,35.278036,0.987412,58.294023,41.959674,0.974412,Random Forest,1
9,27.689122,21.030195,0.996395,32.589210,24.811842,0.992003,XGBoost,1


In [79]:
# Mostrar los mejores pipelines por cluster
print("\nPipelines del mejor proceso por cluster:")
for cluster, pipeline in best_pipelines.items():
    print(f"Cluster {cluster}: {pipeline}")


Pipelines del mejor proceso por cluster:
Cluster 0: Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selector',
                 SelectFromModel(estimator=RandomForestRegressor(random_state=42),
                                 threshold='median')),
                ('regressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=0.8, device=None,
                              early_stopping_rounds=None,
                              enable_categorical=False,...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
      

## Validacion de los pipelines

In [80]:
best_pipelines[0]

Pipeline(steps=[('scaler', StandardScaler()),
                ('feature_selector',
                 SelectFromModel(estimator=RandomForestRegressor(random_state=42),
                                 threshold='median')),
                ('regressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=0.8, device=None,
                              early_stopping_rounds=None,
                              enable_categorical=False,...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=3, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=100, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

In [81]:
combined_df_with_clusters = data_complete.merge(clustered_data[['series_id', 'Cluster']], on='series_id', how='left')
combined_df_with_clusters = combined_df_with_clusters.reset_index(drop=True)
    
# Convertir la columna de fecha a formato datetime y establecerla como índice
combined_df_with_clusters["Date/Time"] = pd.to_datetime(combined_df_with_clusters["Date/Time"])
combined_df_with_clusters.set_index("Date/Time",inplace=True)

In [82]:
columnas=['series_id','Cluster', 'Electricity:Facility [kW](Hourly)',
       'Fans:Electricity [kW](Hourly)', 'Cooling:Electricity [kW](Hourly)',
       'Heating:Electricity [kW](Hourly)',
       'InteriorLights:Electricity [kW](Hourly)',
       'InteriorEquipment:Electricity [kW](Hourly)']

In [83]:
combined_df_with_clusters=combined_df_with_clusters.loc[:,columnas]
combined_df_with_clusters.head()

,series_id,Cluster,Electricity:Facility [kW](Hourly),Fans:Electricity [kW](Hourly),Cooling:Electricity [kW](Hourly),Heating:Electricity [kW](Hourly),InteriorLights:Electricity [kW](Hourly),InteriorEquipment:Electricity [kW](Hourly)
Date/Time,,,,,,,,
2004-01-01 01:00:00,series_1,0,22.453919,3.998243,0.000733,0.0,4.589925,8.1892
2004-01-01 02:00:00,series_1,0,14.637149,0.000000,0.000000,0.0,1.529975,7.4902
2004-01-01 03:00:00,series_1,0,14.651183,0.000000,0.000000,0.0,1.529975,7.4902
2004-01-01 04:00:00,series_1,0,14.657947,0.000000,0.000000,0.0,1.529975,7.4902
2004-01-01 05:00:00,series_1,0,14.806050,0.000000,0.000000,0.0,1.529975,7.4902


In [84]:
# Evaluar todas las series de tiempo en los pipelines
results = evaluate_all_clusters(
    data=combined_df_with_clusters,  # DataFrame con todas las series y sus etiquetas de cluster
    pipelines=best_pipelines,  # Pipelines de los mejores modelos por cluster
    target="Electricity:Facility [kW](Hourly)"  # Variable objetivo
)

Evaluating Cluster 0
Evaluating Cluster 1
Evaluating Cluster 2
Evaluating Cluster 3


In [85]:
# Mostrar resultados por cluster
print("Resultados por Cluster:")
for cluster, metrics in results.items():
    print(f"{cluster}: Average RMSE = {metrics['Average RMSE']:.2f}, Average sMAPE = {metrics['Average sMAPE (%)']:.2f}%")

Resultados por Cluster:
Cluster 0: Average RMSE = 319.86, Average sMAPE = 126.99%
Cluster 1: Average RMSE = 407.50, Average sMAPE = 71.21%
Cluster 2: Average RMSE = 648.44, Average sMAPE = 91.52%
Cluster 3: Average RMSE = 6.75, Average sMAPE = 3.28%


In [86]:
def get_pipeline_features(pipeline):
    """
    Obtiene las características de entrada utilizadas por un pipeline entrenado.

    Parámetros:
    - pipeline: Pipeline entrenado.

    Retorna:
    - features: Lista de características de entrada utilizadas por el pipeline.
    """
    for step_name, step in pipeline.steps:
        # Buscar un selector de características (puede ser personalizado o una clase de sklearn)
        if hasattr(step, 'selected_features_'):
            return step.selected_features_
        # Verificar si es un transformador que no cambia las columnas (ej. StandardScaler)
        elif hasattr(step, 'get_feature_names_out'):
            return step.get_feature_names_out()
    
    # Si ningún paso tiene información sobre características, devolver None
    return None


In [87]:
# Supongamos que tienes un pipeline entrenado para el cluster 0
pipeline = best_pipelines[0]

# Obtener características de entrada utilizadas por el pipeline
features_used = get_pipeline_features(pipeline)

if features_used is not None:
    print("Características utilizadas por el pipeline:", features_used)
else:
    print("El pipeline no incluye un selector explícito de características.")


Características utilizadas por el pipeline: ['Fans:Electricity [kW](Hourly)' 'Cooling:Electricity [kW](Hourly)'
 'Heating:Electricity [kW](Hourly)'
 'InteriorLights:Electricity [kW](Hourly)'
 'InteriorEquipment:Electricity [kW](Hourly)' 'hour' 'day_of_week' 'month']
